In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ..

/home/breno/Documents/master_projects


In [3]:
# !pip3 install yfinance --user
# !pip3 install pandas-datareader --user
# !git clone  https://github.com/jperla/sentiment-data.git

In [3]:
import data_prep

[nltk_data] Downloading package punkt to /home/breno/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
tickers_list = ['DJI']

prices_list = data_prep.get_multiple_stock_prices_yf(tickers_list, key = 'T2MK2AQSBF5UAKF6') # , 

[*********************100%***********************]  1 of 1 completed
Concluded DJI


In [8]:
date_grid = prices_list['DJI'].index

In [9]:
import pandas as pd 

negative = pd.read_csv('sentiment-data/finance/LoughranMcDonald_Negative.csv',header = None)
positive = pd.read_csv('sentiment-data/finance/LoughranMcDonald_Positive.csv',header = None)

stopwords = pd.read_csv('sentiment-data/finance/stopwords.csv',header = None)
stopwords = list(stopwords[0].str.lower())

negative = list(negative[0].str.lower())
positive = list(positive[0].str.lower())

news = data_prep.get_bloomberg_data_from_drive ()
news = data_prep.preprocess_text(news)
news = data_prep.obtain_polarities(news, negative, positive, stopwords)
news = data_prep.get_fixed_date(news, date_grid)
news = news.reset_index()

In [61]:
news['polarity'] = 0.9*(news['polarity']-min(news['polarity']))/(max(news['polarity'])-min(news['polarity'])) + 0.1

In [47]:
c = news.groupby('datetime').created_at.count()

corona_ratio = news.groupby('datetime').created_at.mean()


def calculate_hesitant (row):
    return (1 - row['polarity'])**(1/c[row['datetime']])

news['hesitant_polarity']  = news.apply(calculate_hesitant, axis = 1) 

In [197]:
# !pip3 install -U pyFTS

x = prices_list['DJI']
y = news.groupby('datetime').mean()

prices_list['DJI'] = (pd.DataFrame(x).merge(y, left_index=True, right_index=True) ) #, right_on='datetime'))